<a href="https://colab.research.google.com/github/Lakshyadeepsingh18/Interndesire_projects/blob/main/Sentiment_Analysis_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TWITTER SENTIMENT ANALYSIS PREDICTION.

In [ ]:
from ast import increment_lineno
#import dependencies
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import re
import string
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
#loading

In [ ]:
data=pd.read_csv('/content/twitter_training.csv',header=None,index_col=0)
data.head()

,1,2,3
0,,,
2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2401,Borderlands,Positive,im coming on borderlands and i will murder you...
2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
data=data[[2,3]].reset_index(drop=True)

In [ ]:
data.columns=['Sentiment','Text']
data.head()

,Sentiment,Text
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
data.sample(20)

,Sentiment,Text
72874,Positive,com We have the @nvidia Shield TV Pro Beta and...
65438,Neutral,Johnson & Johnson goes out of business
1931,Neutral,ON BORDERLANDS TWO W USED FOR TO DRAGON SOMEON...
55793,Negative,So wait. I can't install @CallofDuty on my ser...
38965,Irrelevant,This was the first event my company ever repor...
27917,Negative,You know I get it people have great aim on their
21084,Negative,@CSGO thank you for you awesome anticheat... H...
56701,Negative,@ Rainbow6Game why is the option to leave game...
16339,Negative,I admit nothing but my sadness that you are gone
38131,Neutral,I talk way too much shit when I play Hearthsto...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  74682 non-null  object
 1   Text       73996 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [ ]:
data.isnull().sum()

,0
Sentiment,0
Text,686


In [ ]:
data.sample(10)
data.dropna(inplace=True)


In [ ]:
sum(data['Text'].apply(len)>5),sum(data['Text'].apply(len)<=5)

(71890, 2106)

In [ ]:
data=data[data['Text'].apply(len)>5]
print(data.shape)

(71890, 2)


In [ ]:
#feature extraction

In [ ]:
data.duplicated().sum()

np.int64(2646)

In [ ]:
data=data.drop_duplicates()

In [ ]:
data['Sentiment'].value_counts()


,count
Sentiment,
Negative,21082
Positive,18978
Neutral,16981
Irrelevant,12203


In [ ]:
data['Sentiment']=data['Sentiment'].replace('Irrelevant','Neutral')

In [ ]:
data=data.drop_duplicates()

In [ ]:
data.duplicated().sum()

np.int64(0)

In [ ]:
def clean_text(Text):
  if isinstance(Text,str):
    Text=Text.lower()
    Text=re.sub(r'http\S+|www\s','',Text)
    Text=re.sub(r'@\w+','',Text)
    Text=re.sub(r'#\w+','',Text)
    Text=re.sub(r'[^\w\s]','',Text)
    Text=re.sub(r'\d+','',Text)
    Text=re.sub(r'\s+','',Text)
    return Text



In [ ]:
data['clean_text']=data['Text'].apply(clean_text)

In [ ]:
x=data['clean_text']
y=data['Sentiment']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
vectorizer=TfidfVectorizer(max_features=5000)
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)



In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

In [ ]:
#train a model using logistic regression
model=LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [ ]:
#prediction
y_predict=model.predict(x_test)
print(y_predict)

[1 1 1 ... 1 1 1]


In [ ]:
print(y_test)

[2 1 0 ... 1 1 1]


In [ ]:
print("Accuracy:",accuracy_score(y_test,y_predict))
print("\nClassification Report:\n",classification_report(y_test,y_predict))
print("\nConfusion Matrics:\n",confusion_matrix(y_test,y_predict))


Accuracy: 0.4282825364726275

Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.01      0.02      4195
           1       0.42      1.00      0.59      5813
           2       0.92      0.02      0.04      3838

    accuracy                           0.43     13846
   macro avg       0.72      0.34      0.22     13846
weighted avg       0.68      0.43      0.27     13846


Confusion Matrics:
 [[  45 4149    1]
 [   5 5802    6]
 [   5 3750   83]]


In [ ]:
test_data_accuracy=accuracy_score(y_test,y_predict)

In [ ]:
print("accuracy score",test_data_accuracy)

accuracy score 0.4282825364726275


42.8% accuracy

SAVING THE TEXT MODEL

In [ ]:
import pickle
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using the saved model for future prediction.


In [ ]:
labeled_model=pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
x_new=x_test[34]
print(y_predict[34])
prediction=labeled_model.predict(x_new)
print(prediction)
if(prediction[0]==0):
  print('negative')
else:
  print("positive")

1
[1]
positive
